# Scrape human made images

This code is designed to scrape images from Google search results. It utilizes Selenium and Chromedriver to automate the process of visiting search result pages, scrolling down to load more images, and extracting the image URLs. The images are then downloaded and saved into specific category folders.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
import os
import urllib.request
import ssl

# Set up directories
HUMAN = '/Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/Human'

# Specify the path to the ChromeDriver
CHROME_DRIVER_PATH = '/Users/sakai/Downloads/chromedriver-mac-arm64-2/chromedriver'


# URLs to visit
urls = ['https://www.google.com/search?q=art+graphic+design&tbm=isch&ved=2ahUKEwisiM6s2pr-AhXPGTQIHXYSB84Q2-cCegQIABAA&oq=art+graphic+design&gs_lcp=CgNpbWcQAzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDoECCMQJ1DxAliJCmDFDmgAcAB4AIABb4gB1AWSAQM3LjGYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=bZYxZOydC8-z0PEP9qSc8Aw&bih=609&biw=1280',
       'https://www.google.com/search?q=anime+art&tbm=isch&ved=2ahUKEwjdz9iv2pr-AhUyAzQIHXdzAjMQ2-cCegQIABAA&oq=anime+art&gs_lcp=CgNpbWcQAzIKCAAQigUQsQMQQzIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDoHCAAQigUQQzoECCMQJ1DjBFinDWDoD2gAcAB4AIABbIgBvgeSAQM3LjOYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=c5YxZJ3KJ7KG0PEP9-aJmAM&bih=609&biw=1280',
       'https://www.google.com/search?q=people+outside&tbm=isch&ved=2ahUKEwid5LC33pr-AhX0HzQIHWLOAkMQ2-cCegQIABAA&oq=people+outside&gs_lcp=CgNpbWcQAzIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQ6BAgjECc6BwgAEIoFEEM6CggAEIoFELEDEEM6CAgAEIAEELEDOggIABCxAxCDAToGCAAQCBAeUMsIWNkrYJUsaARwAHgAgAFtiAHPDJIBBDE2LjKYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=tZoxZN21H_S_0PEP4pyLmAQ&bih=609&biw=1280',
       'https://www.google.com/search?q=digital+art&tbm=isch&ved=2ahUKEwjejJ7B3pr-AhWLATQIHQqgCDMQ2-cCegQIABAA&oq=digital+art&gs_lcp=CgNpbWcQAzIKCAAQigUQsQMQQzIHCAAQigUQQzIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEOgQIIxAnOggIABCABBCxAzoICAAQsQMQgwFQlgVY7Q9guRRoAHAAeACAAWmIAdgIkgEEMTAuMpgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=ypoxZN7_CouD0PEPisCimAM&bih=609&biw=1280',
       'https://www.google.com/search?q=digital+art&tbm=isch&chips=q:digital+art,g_1:portrait:rxaMKhJ3aNU%3D&hl=en&sa=X&ved=2ahUKEwiazeWH35r-AhUTJ30KHXJpCOoQ4lYoBHoECAEQLg&biw=1263&bih=609',
       'https://www.google.com/search?q=digital+art&tbm=isch&chips=q:digital+art,g_1:drawing:wv9WUjQunSk%3D&hl=en&sa=X&ved=2ahUKEwiazeWH35r-AhUTJ30KHXJpCOoQ4lYoBnoECAEQMg&biw=1263&bih=609',
       'https://www.google.com/search?q=abstract+art&tbm=isch&ved=2ahUKEwixmvia35r-AhXMBTQIHWKLBCkQ2-cCegQIABAA&oq=abstract+art&gs_lcp=CgNpbWcQAzIKCAAQigUQsQMQQzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDoECCMQJ1CwBFjxD2CPEmgAcAB4AIABaYgBowmSAQQxMS4ymAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=hpsxZLG_EsyL0PEP4paSyAI&bih=609&biw=1263&hl=en',
       'https://www.google.com/search?q=paintings&tbm=isch&ved=2ahUKEwjirOTF35r-AhULHDQIHQXqChwQ2-cCegQIABAA&oq=paintings&gs_lcp=CgNpbWcQAzIKCAAQigUQsQMQQzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDoECCMQJ1C9DVi1EmDwE2gAcAB4AIABqAGIAeMIkgEDNS41mAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=4JsxZOK8CYu40PEPhdSr4AE&bih=609&biw=1263&hl=en',
       'https://www.google.com/search?q=cartoon&tbm=isch&ved=2ahUKEwid9c-W4Jr-AhVgADQIHf3LAdYQ2-cCegQIABAA&oq=cartoon&gs_lcp=CgNpbWcQAzIKCAAQigUQsQMQQzIHCAAQigUQQzIKCAAQigUQsQMQQzIKCAAQigUQsQMQQzIHCAAQigUQQzIHCAAQigUQQzIKCAAQigUQsQMQQzIKCAAQigUQsQMQQzIHCAAQigUQQzIICAAQgAQQsQM6BAgjECc6BQgAEIAEUMgEWOwMYKMOaABwAHgAgAFviAHjBZIBAzcuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=iZwxZN2MKuCA0PEP_ZeHsA0&bih=609&biw=1263&hl=en',
       'https://www.google.com/search?q=animals&tbm=isch&ved=2ahUKEwixnMK34Jr-AhWKBzQIHdT6AE8Q2-cCegQIABAA&oq=animals&gs_lcp=CgNpbWcQAzIICAAQgAQQsQMyCwgAEIAEELEDEIMBMgUIABCABDIFCAAQgAQyCAgAEIAEELEDMgUIABCABDIICAAQgAQQsQMyBQgAEIAEMgUIABCABDIFCAAQgAQ6BAgjECc6CggAEIoFELEDEEM6BwgAEIoFEENQrAVY6gpg9wtoAHAAeACAAW2IAeMFkgEDNy4xmAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=zpwxZLH9KIqP0PEP1PWD-AQ&bih=609&biw=1263&hl=en',
       'https://www.google.com/search?q=drawings&tbm=isch&ved=2ahUKEwixnMK34Jr-AhWKBzQIHdT6AE8Q2-cCegQIABAA&oq=drawings&gs_lcp=CgNpbWcQAzIKCAAQigUQsQMQQzIHCAAQigUQQzIHCAAQigUQQzIKCAAQigUQsQMQQzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzIHCAAQigUQQzoECCMQJzoFCAAQgAQ6CAgAEIAEELEDUPYEWKEMYOwMaABwAHgAgAFyiAH1BZIBAzcuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=zpwxZLH9KIqP0PEP1PWD-AQ&bih=609&biw=1263&hl=en',
        'https://www.google.com/search?q=people+doing+things+real&tbm=isch&ved=2ahUKEwjQ-9ST95r-AhXhGTQIHWqpDgEQ2-cCegQIABAA&oq=people+doing+things+real&gs_lcp=CgNpbWcQAzIGCAAQCBAeOgQIIxAnOgcIABCKBRBDOgUIABCABDoGCAAQBRAeUIEDWNcHYKQJaABwAHgAgAGoAYgBsQWSAQMyLjSYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=obQxZJCnHuGz0PEP6tK6CA&bih=609&biw=1280',
        'https://www.google.com/search?q=portraits&tbm=isch&chips=q:portraits,g_1:photography:G7hcdP5jhN0%3D&hl=en&sa=X&ved=2ahUKEwj9tra295r-AhW2GTQIHYkLApAQ4lYoAnoECAEQKg&biw=1263&bih=609',
        'https://www.google.com/search?q=watercolour&tbm=isch&ved=2ahUKEwiDrJ6v_5r-AhURAzQIHWRjBnIQ2-cCegQIABAA&oq=watercolour&gs_lcp=CgNpbWcQAzIICAAQgAQQsQMyCAgAEIAEELEDMggIABCABBCxAzIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQ6BAgjECc6BwgAEIoFEENQ5QZYjw9glhBoAHAAeACAAW-IAcQGkgEDNy4ymAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=Pr0xZIOVMZGG0PEP5MaZkAc&bih=609&biw=1263&hl=en',
        'https://www.google.com/search?q=oil+painting&tbm=isch&ved=2ahUKEwi5l56y_5r-AhX6HDQIHQlkBZ4Q2-cCegQIABAA&oq=oi&gs_lcp=CgNpbWcQARgAMgoIABCKBRCxAxBDMgcIABCKBRBDMgcIABCKBRBDMgoIABCKBRCxAxBDMggIABCABBCxAzIICAAQgAQQsQMyCAgAEIAEELEDMggIABCABBCxAzIICAAQgAQQsQMyBQgAEIAEOgQIIxAnOggIABCxAxCDAVDECljhC2CiF2gAcAB4AIABY4gBkAKSAQEzmAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=Rb0xZPngBfq50PEPiciV8Ak&bih=609&biw=1263&hl=en'
       ]

# Categories for images
categories = [
    'a_graphic_deisgn_art', 'b_anime_art', 'c_people_outside', 'd_digital_art',
    'e_digital_art_portrait', 'f_igital_art_drawing', 'g_abstract_art', 'h_paintings', 'i_cartoon',
    'j_animals', 'k_drawings', 'l_people_doing_things','m_real_portraits', 'n_watercolour', 'o_oil'
]

# Bỏ qua SSL verification
context = ssl._create_unverified_context()

chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("--headless")  # Runs Chrome in headless mode.
chrome_service = Service(CHROME_DRIVER_PATH)

# Initialize the browser driver
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# Tải và lưu ảnh từ mỗi URL
for idx, url in enumerate(urls):
    driver.get(url)
    time.sleep(2)  # Đợi trang tải xong

    # Cuộn trang để tải ảnh
    body = driver.find_element(By.TAG_NAME, 'body')
    for _ in range(10):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    # Tìm tất cả các ảnh sau khi cuộn
    images = driver.find_elements(By.TAG_NAME, 'img')
    os.makedirs(os.path.join(HUMAN, categories[idx]), exist_ok=True)

    # Duyệt và tải từng ảnh
    for i, img in enumerate(images):
        src = img.get_attribute('src')

        # Kiểm tra thêm thuộc tính 'data-src' nếu 'src' rỗng
        if not src:
            src = img.get_attribute('data-src')

        if src and 'http' in src:
            try:
                file_path = os.path.join(HUMAN, categories[idx], f'image_{i}.jpg')

                # Tải ảnh với urlopen và bỏ qua xác minh SSL
                with urllib.request.urlopen(src, context=context) as response, open(file_path, 'wb') as out_file:
                    out_file.write(response.read())

                print(f"Đã tải xong ảnh {i} từ '{categories[idx]}' - Lưu tại: {file_path}")

            except Exception as e:
                print(f"Lỗi khi tải ảnh {i}: {e}")

driver.quit()


# Scrape DALLE images

This code is used to scrape images from the website dalle2.gallery, which hosts AI-generated images created by DALLE. It automates the process of visiting the website, scrolling down to load more images, and downloading the unique image URLs until the specified maximum number of images (max_images) is reached. The downloaded images are saved with sequential numbers as filenames.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
import os
import urllib.request
import ssl

# Specify the path to the ChromeDriver
CHROME_DRIVER_PATH = '/Users/sakai/Downloads/chromedriver-mac-arm64-2/chromedriver'
urls = ['https://dalle2.gallery/#search-random']
DALLE = '/Users/sakai/VIET_Working/STUDY_WORK/Ky5/Python/DALLE'

# Bỏ qua SSL verification
context = ssl._create_unverified_context()

chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("--headless")  # Runs Chrome in headless mode.
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--allow-insecure-localhost')
chrome_options.add_argument('--disable-web-security')
chrome_options.add_argument('--ignore-ssl-errors=yes')
chrome_service = Service(CHROME_DRIVER_PATH)

# Initialize the browser driver
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# # Tải và lưu ảnh từ mỗi URL
for idx, url in enumerate(urls):
    driver.get(url)
    print('tai trang xong')
    time.sleep(2)  # Đợi trang tải xong

    # Cuộn trang để tải ảnh
    body = driver.find_element(By.TAG_NAME, 'body')
    for _ in range(10):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    # Tìm tất cả các ảnh sau khi cuộn
    images = driver.find_elements(By.TAG_NAME, 'img')

    print(len(images))

    # Duyệt và tải từng ảnh
    for i, img in enumerate(images):
        src = img.get_attribute('src')
        
        print(f'duyet va tai tung anh {i}')

        # Kiểm tra thêm thuộc tính 'data-src' nếu 'src' rỗng
        if not src:
            src = img.get_attribute('data-src')

        if src and 'http' in src:
            try:
                file_path = os.path.join(DALLE, f'image_{i}.jpg')

                # Tải ảnh với urlopen và bỏ qua xác minh SSL
                with urllib.request.urlopen(src, context=context) as response, open(file_path, 'wb') as out_file:
                    out_file.write(response.read())

                print(f"Đã tải xong ảnh {i} - Lưu tại: {file_path}")

            except Exception as e:
                print(f"Lỗi khi tải ảnh {i}: {e}")

driver.quit()